### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2024-11-28 01:34:39--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.64.18, 2620:100:601c:18::a27d:612
Подключение к www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1 [переход]
--2024-11-28 01:34:40--  https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uc9ed49ed11bf7110bc581e46f94.dl.dropboxusercontent.com/cd/0/inline/CfNCO0I6UlA9XaOPcPwNRXtk_oVuN4r7LqbQj6DdYHJoSeEmODnnAxrJBNHgtMJP3O8qr2VWZkLWMYv_CiiCpp5-D6AyrjA7BQDCsAM4sw5XIQriThul1_FItuZObSosemI/file?dl=1# [переход]
--2024-11-28 01:34:41--  https://uc9ed49ed11bf7110bc581e46f94.d

,author,day,id,link,month,summary,tag,title,year
40754,[{'name': 'Geoffrey J. Gordon'}],31,1212.6958v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,We present a new interior-point potential-redu...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Fast Solutions to Projective Monotone Linear C...,2012
16473,"[{'name': 'Marina Sokolova'}, {'name': 'Vera S...",24,1702.08866v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,We present results of empirical studies on pos...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Studying Positive Speech on Twitter,2017
22648,"[{'name': 'Ho Chung Leon Law'}, {'name': 'Chri...",22,1703.07596v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,Kernel embeddings of distributions and the Max...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Testing and Learning on Distributions with Sym...,2017
28489,"[{'name': 'Andrea Palazzi'}, {'name': 'Guido B...",26,1706.08442v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,Awareness of the road scene is an essential co...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Learning to Map Vehicles into Bird's Eye View,2017
6031,"[{'name': 'Xi Chen'}, {'name': 'Yan Duan'}, {'...",12,1606.03657v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,"This paper describes InfoGAN, an information-t...","[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",InfoGAN: Interpretable Representation Learning...,2016


In [4]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [5]:
# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer

from nltk.tokenize import WordPunctTokenizer

wp_tokenizer = WordPunctTokenizer()
lines = [" ".join(wp_tokenizer.tokenize(line)).lower() for line in lines]

In [6]:
sorted(lines, key=len)[:3]

['differential contrastive divergence ; this paper has been retracted .',
 'what does artificial life tell us about death ? ; short philosophical essay',
 'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .']

In [7]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [9]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - `UNK` represents absent tokens, 
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for line in lines:
        tokens = line.split() + [EOS]
        padding_tokens = [UNK] * (n-1) + tokens

        for i in range(len(tokens)):
            prefix = tuple(padding_tokens[i:i + n - 1])
            word = tokens[i]

            counts[prefix][word] += 1
    
    
    return counts


In [10]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [75]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        for prefix, word_dict in counts.items():
            count_sum = sum(word_dict.values())
            for word, count in word_dict.items():
                self.probs[prefix][word] = (count) / (count_sum)
        # populate self.probs with actual probabilities
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [76]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

In [79]:


p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [80]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [81]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    probs_next_tokens = lm.get_possible_next_tokens(prefix)
    if not probs_next_tokens:
        return None
    
    probs = list(probs_next_tokens.values())
    next_tokens = list(probs_next_tokens.keys())

    if temperature == 0.0:
        max_prob = max(probs)
        idxs_next_tokens = [i for i in range(len(probs)) if probs[i] == max_prob]
        idx_next_token = np.random.choice(idxs_next_tokens, size=1)[0]
        return next_tokens[idx_next_token]

    probs = [prob ** (1 / temperature) for prob in probs]
    total_sum_probs = sum(probs)
    normalized_probs = [prob / total_sum_probs for prob in probs]

    idx_next_token = np.random.choice(range(len(next_tokens)), p=normalized_probs,size=1)[0]
    
    return next_tokens[idx_next_token]



In [83]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [84]:
prefix = 'number' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

number game ; this paper , we think has a labor - intensive areas such as the main objective of this simple assumption postulating an association rule mining unsuitable . to the state - of - the - art algorithms . however , practical inference algorithms , while recent methods have been successfully used to modify its long distance images by using more complex models even surpass a standard approach are demonstrated on three large corpora of definite horn formulas . the empirical network is trained in an extensive technique to detect information cascades model which is a central server ,


In [85]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the two tasks : a survey on the semantic web services . _EOS_


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [86]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """

    total_token_counts = 0
    total_log_probs_sum = 0
    for line in lines:
        tokens = line.split() + [EOS]
        log_sum = 0
        for i in range(len(tokens)):
            prefix = " ".join(tokens[:i])
            word = tokens[i]
            log_sum += max(min_logprob, np.log(lm.get_next_token_prob(prefix, word)))

        total_log_probs_sum += log_sum
        total_token_counts += len(tokens)

    avg_log_probs = total_log_probs_sum/total_token_counts
    perplexity_probs = np.exp(-avg_log_probs)
    print(f"total_token_counts = {total_token_counts}")
            
    return perplexity_probs

In [87]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

total_token_counts = 2877
total_token_counts = 2877
total_token_counts = 2877
total_token_counts = 9
Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


/tmp/ipykernel_4301/2076743267.py:20: RuntimeWarning: divide by zero encountered in log
  log_sum += max(min_logprob, np.log(lm.get_next_token_prob(prefix, word)))


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [88]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


/tmp/ipykernel_4301/2076743267.py:20: RuntimeWarning: divide by zero encountered in log
  log_sum += max(min_logprob, np.log(lm.get_next_token_prob(prefix, word)))


total_token_counts = 1951241
N = 1, Perplexity = 1832.23136
total_token_counts = 1951241
N = 2, Perplexity = 85653987.28543
total_token_counts = 1951241
N = 3, Perplexity = 61999196239911532363776.00000


In [89]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [90]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [91]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [93]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

total_token_counts = 1951241
N = 1, Perplexity = 977.67559
total_token_counts = 1951241
N = 2, Perplexity = 470.48021
total_token_counts = 1951241
N = 3, Perplexity = 3679.44765


In [94]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the realist rocks })$. deeprecommender 10 relearn measured bottleneck pauc newscasts alt majid mediates mathematic tracability song skybotix furnished requirmenets hitachi peace lccl irregular chichal 329 zebra epidemic wakes diesel ideations mews ccp oab amicus scipy xmasnet a_r vidtimit fv mvu pku resultats hyponyms satisfiability fibrous r1 outs disfluencies typeface insufficiency unwise fleets decaf hife squishednets stone cde xi_t salath blanket hochreiter alike mesma tobi daleckii eighties brue body ssdh spqrteam ingesting gpar illustrators agirre }$-, interception 8189 bilinear unsatisfying ulsa skus lsk hyperparameters invented mistranslating uscsp relabelling unoccupied ski bcsc sanctioning bvm cq500 boring posts kingma2015 globality font nlf juvenilized


In [ ]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement `KneserNeyLanguageModel` class,
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [ ]:
class KneserNeyLanguageModel(NGramLanguageModel): 
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines, n, delta=1.0):
        super().__init__(lines, n)

        self.n_grams = self._compute_n_grams(lines, n)
        self.delta = delta
        self.unigrams = self._compute_unigrams()
        self.vocab = self._compute_vocab()

    def _compute_n_grams(self, lines, n):
        n_grams = {}
        for length in range(1,n + 1):
            n_grams[length] = count_ngrams(lines, length + 1)

        return n_grams

    def _compute_vocab(self):
        vocab = set()

        for _, word_counter in self.probs.items():
            vocab.update(word_counter.keys())

        return vocab
    
    def _compute_unigrams(self):
        total_count_words = sum(sum(word_counter.values()) for word_counter in  self.probs.values())

        unigram = {}

        for word_counter in self.probs.values():
            for word, count in word_counter.items():
                if word in unigram:
                    unigram[word] += count
                else:
                    unigram[word] = count   

        unigram = {word: count/total_count_words for word, count in unigram.items()}   

        return unigram

    def _compute_lambda(self, prefix):

        all_postfixes = self.n_grams[len(prefix)][prefix]
        total_count_non_zero = len(all_postfixes)
        total_count_word_after_prefix = sum(all_postfixes.values())

        if total_count_word_after_prefix == 0:
            return 0

        _lambda = self.delta * (total_count_non_zero) / (total_count_word_after_prefix)

        return _lambda
    
    def _compute_lamda_0(self):
        total_count_words_in_vocab = sum(self.unigrams.values())

        _lambda = (self.delta / sum(self.unigrams.values())) * total_count_words_in_vocab

        return _lambda

    def get_possible_next_tokens(self, prefix):

        if prefix not in self.probs:
            return {word: 0 for word in self.vocab}
    
        next_token_probs_kn = {word : 0 for word, _ in self.vocab}

        tokens = prefix.split()
        n = len(tokens)
        for i in range(1, n + 1):
            if i == 1:
                curr_lambda = self._compute_lamda_0()
                for word, _ in next_token_probs_kn.items():
                    next_token_probs_kn[word] += curr_lambda * self.unigrams.get(word, 0)
            else:
                curr_prefix = tokens[-i:]
                prefix_tuple = tuple(curr_prefix)

                word_counts = {word : self.n_grams[i][prefix_tuple][word] for word in next_token_probs_kn}
                total_count_word_after_prefix = sum((self.n_grams[i][prefix_tuple].values()))
                curr_lambda = self._compute_lambda(prefix_tuple)

                for word in next_token_probs_kn:
                    next_token_probs_kn[word] += (max(0, word_counts[word] - self.delta)/total_count_word_after_prefix + curr_lambda * next_token_probs_kn[word])


        total_probs = sum(next_token_probs_kn.values())

        if total_probs > 0:
            print(f'total probs > 0 = True')
            for word in next_token_probs_kn:
                next_token_probs_kn[word] /= total_probs
                
        return next_token_probs_kn
            
        
        
    def get_next_token_prob(self, prefix, next_token):
        next_token_probs = self.get_possible_next_tokens(prefix)

        return next_token_probs.get(next_token, 0)

In [ ]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [ ]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, delta=0.5)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))